In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.1`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.$
import org.apache.log4j.{Level, Logger}

In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

import org.apache.spark.sql._
import org.apache.spark.sql.functions._

In [3]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("UDF")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
23/08/20 14:13:22 WARN Utils: Your hostname, Air.local resolves to a loopback address: 127.0.0.1; using 192.168.88.21 instead (on interface en5)
23/08/20 14:13:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/08/20 14:13:22 INFO SparkContext: Running Spark version 3.4.1
23/08/20 14:13:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/20 14:13:23 INFO ResourceUtils: ==============================================================
23/08/20 14:13:23 INFO ResourceUtils: No custom resources configured for spark.driver.
23/08/20 14:13:23 INFO ResourceUtils: ==============================================================
23/08/20 14:13:23 INFO SparkContext: Submitted application: UDF
23/08/20 14:13:23 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount

spark: SparkSession = org.apache.spark.sql.SparkSession@31af882
import spark.implicits._

In [4]:
val data = Seq((1, "Michael", 3000), (2, "Andy", 4500), (3, "Justin", 3500), (4, "Berta", 4000)).toDF("id", "name", "salary")
data.show()

23/08/20 14:13:26 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
23/08/20 14:13:26 INFO SharedState: Warehouse path is 'file:/Users/vadim/Projects/OtusTeam/SparkDeveloper/lesson-07/Notebooks/spark-warehouse'.
23/08/20 14:13:27 INFO CodeGenerator: Code generated in 209.7295 ms
23/08/20 14:13:28 INFO CodeGenerator: Code generated in 18.17575 ms
23/08/20 14:13:28 INFO CodeGenerator: Code generated in 8.740292 ms


+---+-------+------+
| id|   name|salary|
+---+-------+------+
|  1|Michael|  3000|
|  2|   Andy|  4500|
|  3| Justin|  3500|
|  4|  Berta|  4000|
+---+-------+------+



data: DataFrame = [id: int, name: string ... 1 more field]

## UDF без аргументов

In [5]:
val random = udf(() => java.lang.Math.random())
data.withColumn("random", random()).show

23/08/20 14:13:28 INFO CodeGenerator: Code generated in 5.437417 ms
23/08/20 14:13:28 INFO CodeGenerator: Code generated in 5.60875 ms
23/08/20 14:13:28 INFO CodeGenerator: Code generated in 7.062458 ms


+---+-------+------+-------------------+
| id|   name|salary|             random|
+---+-------+------+-------------------+
|  1|Michael|  3000|0.07767742749372786|
|  2|   Andy|  4500| 0.5883059631817982|
|  3| Justin|  3500|0.06464612763206401|
|  4|  Berta|  4000|0.32936604102493616|
+---+-------+------+-------------------+



random: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cell5$Helper$$Lambda$4791/1391364260@2ec5a723,
  DoubleType,
  List(),
  Some(
    ExpressionEncoder(
      BoundReference(0, DoubleType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, DoubleType),
          DoubleType,
          List("- root class: \"double\"")
        ),
        List("- root class: \"double\"")
      ),
      Double
    )
  ),
  None,
  false,
  true
)

## UDF с одним аргументом

In [6]:
val plusOne = udf((x: Int) => x + 1)
data.select($"name", $"salary", plusOne($"salary").as("new_salary")).show

23/08/20 14:13:29 INFO CodeGenerator: Code generated in 13.430625 ms
23/08/20 14:13:29 INFO CodeGenerator: Code generated in 12.229083 ms
23/08/20 14:13:29 INFO CodeGenerator: Code generated in 10.158833 ms


+-------+------+----------+
|   name|salary|new_salary|
+-------+------+----------+
|Michael|  3000|      3001|
|   Andy|  4500|      4501|
| Justin|  3500|      3501|
|  Berta|  4000|      4001|
+-------+------+----------+



plusOne: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cell6$Helper$$Lambda$4867/248097309@4ddb8ae7,
  IntegerType,
  List(
    Some(
      ExpressionEncoder(
        BoundReference(0, IntegerType, false),
        AssertNotNull(
          UpCast(
            GetColumnByOrdinal(0, IntegerType),
            IntegerType,
            List("- root class: \"int\"")
          ),
          List("- root class: \"int\"")
        ),
        Int
      )
    )
  ),
  Some(
    ExpressionEncoder(
      BoundReference(0, IntegerType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, IntegerType),
          IntegerType,
          List("- root class: \"int\"")
        ),
        List("- root class: \"int\"")
      ),
      Int
    )
  ),
  None,
  false,
  true
)

## UDF с двумя аргументами

In [7]:
val strLenPlus = udf((x: String, y: Int) => x.length + y)

strLenPlus: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cell7$Helper$$Lambda$4905/1830569337@565efd4b,
  IntegerType,
  List(
    Some(
      ExpressionEncoder(
        StaticInvoke(
          class org.apache.spark.unsafe.types.UTF8String,
          StringType,
          "fromString",
          List(BoundReference(0, ObjectType(class java.lang.String), true)),
          List(),
          true,
          false,
          true
        ),
        Invoke(
          UpCast(
            GetColumnByOrdinal(0, StringType),
            StringType,
            List("- root class: \"java.lang.String\"")
          ),
          "toString",
          ObjectType(class java.lang.String),
          List(),
          List(),
          true,
          false,
          true
        ),
        java.lang.String
      )
    ),
    Some(
      ExpressionEncoder(
        BoundReference(0, IntegerType, false),
        AssertNotNull(
          UpCast(
            GetColumnByOrdi

In [8]:
data.select($"name", length($"name"), strLenPlus($"name",  $"id").as("nameLenPlusId")).show

23/08/20 14:13:30 INFO CodeGenerator: Code generated in 6.548458 ms
23/08/20 14:13:30 INFO CodeGenerator: Code generated in 5.186625 ms
23/08/20 14:13:30 INFO CodeGenerator: Code generated in 13.507167 ms


+-------+------------+-------------+
|   name|length(name)|nameLenPlusId|
+-------+------------+-------------+
|Michael|           7|            8|
|   Andy|           4|            6|
| Justin|           6|            9|
|  Berta|           5|            9|
+-------+------------+-------------+



In [9]:
data.select($"name", length($"name"), strLenPlus($"name", lit(1)).as("nameLenPlus1")).show

+-------+------------+------------+
|   name|length(name)|nameLenPlus1|
+-------+------------+------------+
|Michael|           7|           8|
|   Andy|           4|           5|
| Justin|           6|           7|
|  Berta|           5|           6|
+-------+------------+------------+



## UDF для выражений WHERE

In [10]:
val oneArgFilter= udf((n: Int) => { n > 2 })
data.where(oneArgFilter($"id")).show

23/08/20 14:13:31 INFO CodeGenerator: Code generated in 78.504375 ms


+---+------+------+
| id|  name|salary|
+---+------+------+
|  3|Justin|  3500|
|  4| Berta|  4000|
+---+------+------+



oneArgFilter: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cell10$Helper$$Lambda$4933/615272381@1643e8d5,
  BooleanType,
  List(
    Some(
      ExpressionEncoder(
        BoundReference(0, IntegerType, false),
        AssertNotNull(
          UpCast(
            GetColumnByOrdinal(0, IntegerType),
            IntegerType,
            List("- root class: \"int\"")
          ),
          List("- root class: \"int\"")
        ),
        Int
      )
    )
  ),
  Some(
    ExpressionEncoder(
      BoundReference(0, BooleanType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, BooleanType),
          BooleanType,
          List("- root class: \"boolean\"")
        ),
        List("- root class: \"boolean\"")
      ),
      Boolean
    )
  ),
  None,
  false,
  true
)

## Производительность

In [11]:
val train = spark.read.option("inferSchema", true).option("header", true).csv("../data/train.csv").cache

23/08/20 14:13:31 INFO InMemoryFileIndex: It took 75 ms to list leaf files for 1 paths.
23/08/20 14:13:31 INFO InMemoryFileIndex: It took 0 ms to list leaf files for 1 paths.
23/08/20 14:13:32 INFO FileSourceStrategy: Pushed Filters: 
23/08/20 14:13:32 INFO FileSourceStrategy: Post-Scan Filters: (length(trim(value#143, None)) > 0)
23/08/20 14:13:33 INFO CodeGenerator: Code generated in 17.823084 ms
23/08/20 14:13:33 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 349.2 KiB, free 2004.3 MiB)
23/08/20 14:13:33 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 2004.2 MiB)
23/08/20 14:13:33 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on 192.168.88.21:62937 (size: 34.1 KiB, free: 2004.6 MiB)
23/08/20 14:13:33 INFO SparkContext: Created broadcast 0 from csv at cell11.sc:1
23/08/20 14:13:33 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as

train: Dataset[Row] = [row_id: int, timestamp: bigint ... 8 more fields]

In [12]:
train.show(5)

23/08/20 14:15:25 INFO CodeGenerator: Code generated in 56.984125 ms
23/08/20 14:15:25 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 349.0 KiB, free 2003.8 MiB)
23/08/20 14:15:25 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 34.0 KiB, free 2003.8 MiB)
23/08/20 14:15:25 INFO BlockManagerInfo: Added broadcast_4_piece0 in memory on 192.168.88.21:62937 (size: 34.0 KiB, free: 2004.5 MiB)
23/08/20 14:15:25 INFO SparkContext: Created broadcast 4 from show at cell12.sc:1
23/08/20 14:15:25 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as scanning 4194304 bytes.
23/08/20 14:15:25 INFO SparkContext: Starting job: show at cell12.sc:1
23/08/20 14:15:25 INFO DAGScheduler: Got job 2 (show at cell12.sc:1) with 1 output partitions
23/08/20 14:15:25 INFO DAGScheduler: Final stage: ResultStage 2 (show at cell12.sc:1)
23/08/20 14:15:25 INFO DAGScheduler: Parents of final st

+------+---------+-------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|row_id|timestamp|user_id|content_id|content_type_id|task_container_id|user_answer|answered_correctly|prior_question_elapsed_time|prior_question_had_explanation|
+------+---------+-------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|     0|        0|    115|      5692|              0|                1|          3|                 1|                       null|                          null|
|     1|    56943|    115|      5716|              0|                2|          2|                 1|                    37000.0|                         false|
|     2|   118363|    115|       128|              0|                0|          0|                 1|                    55000.0|                         false|
|     3|   131167|    115|  

In [13]:
train.count

23/08/20 14:15:56 INFO CodeGenerator: Code generated in 36.559292 ms
23/08/20 14:15:56 INFO DAGScheduler: Registering RDD 23 (count at cell13.sc:1) as input to shuffle 0
23/08/20 14:15:56 INFO DAGScheduler: Got map stage job 3 (count at cell13.sc:1) with 44 output partitions
23/08/20 14:15:56 INFO DAGScheduler: Final stage: ShuffleMapStage 3 (count at cell13.sc:1)
23/08/20 14:15:56 INFO DAGScheduler: Parents of final stage: List()
23/08/20 14:15:56 INFO DAGScheduler: Missing parents: List()
23/08/20 14:15:56 INFO DAGScheduler: Submitting ShuffleMapStage 3 (MapPartitionsRDD[23] at count at cell13.sc:1), which has no missing parents
23/08/20 14:15:56 INFO MemoryStore: Block broadcast_6 stored as values in memory (estimated size 22.5 KiB, free 1946.7 MiB)
23/08/20 14:15:56 INFO MemoryStore: Block broadcast_6_piece0 stored as bytes in memory (estimated size 10.7 KiB, free 1946.7 MiB)
23/08/20 14:15:56 INFO BlockManagerInfo: Added broadcast_6_piece0 in memory on 192.168.88.21:62937 (size: 1

res13: Long = 101230332L

In [14]:
val to_month = udf((ts_ms: Long) => ts_ms / 31536000000L / 12) // 1 year is 31536000000 ms

to_month: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cell14$Helper$$Lambda$6657/2134979872@230db4be,
  LongType,
  List(
    Some(
      ExpressionEncoder(
        BoundReference(0, LongType, false),
        AssertNotNull(
          UpCast(
            GetColumnByOrdinal(0, LongType),
            LongType,
            List("- root class: \"long\"")
          ),
          List("- root class: \"long\"")
        ),
        Long
      )
    )
  ),
  Some(
    ExpressionEncoder(
      BoundReference(0, LongType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, LongType),
          LongType,
          List("- root class: \"long\"")
        ),
        List("- root class: \"long\"")
      ),
      Long
    )
  ),
  None,
  false,
  true
)

In [15]:
val t1 = System.nanoTime

train
    .select($"content_id", to_month($"timestamp").as("month"))
    .groupBy("content_id")
    .mean("month")
    .show()

val duration1 = (System.nanoTime - t1) / 1e9d
duration1

23/08/20 14:17:37 INFO CodeGenerator: Code generated in 46.094125 ms
23/08/20 14:17:37 INFO DAGScheduler: Registering RDD 31 (show at cell15.sc:6) as input to shuffle 1
23/08/20 14:17:37 INFO DAGScheduler: Got map stage job 5 (show at cell15.sc:6) with 44 output partitions
23/08/20 14:17:37 INFO DAGScheduler: Final stage: ShuffleMapStage 6 (show at cell15.sc:6)
23/08/20 14:17:37 INFO DAGScheduler: Parents of final stage: List()
23/08/20 14:17:37 INFO DAGScheduler: Missing parents: List()
23/08/20 14:17:37 INFO DAGScheduler: Submitting ShuffleMapStage 6 (MapPartitionsRDD[31] at show at cell15.sc:6), which has no missing parents
23/08/20 14:17:37 INFO MemoryStore: Block broadcast_8 stored as values in memory (estimated size 52.8 KiB, free 26.8 MiB)
23/08/20 14:17:37 INFO MemoryStore: Block broadcast_8_piece0 stored as bytes in memory (estimated size 23.9 KiB, free 26.8 MiB)
23/08/20 14:17:37 INFO BlockManagerInfo: Added broadcast_8_piece0 in memory on 192.168.88.21:62937 (size: 23.9 KiB,

+----------+----------+
|content_id|avg(month)|
+----------+----------+
|      4519|       0.0|
|      4818|       0.0|
|      5518|       0.0|
|     13285|       0.0|
|     12027|       0.0|
|       833|       0.0|
|      9427|       0.0|
|       496|       0.0|
|      5156|       0.0|
|      6336|       0.0|
|      2866|       0.0|
|      1959|       0.0|
|      7982|       0.0|
|     23336|       0.0|
|       148|       0.0|
|      1342|       0.0|
|      1088|       0.0|
|      1580|       0.0|
|       471|       0.0|
|      2122|       0.0|
+----------+----------+
only showing top 20 rows



t1: Long = 1652190318093666L
duration1: Double = 5.363646375
res15_3: Double = 5.363646375

In [16]:
val t2 = System.nanoTime

train
    .select($"content_id", ($"timestamp" / 31536000000L / 12).alias("month"))
    .groupBy("content_id")
    .mean("month")
    .show()

val duration2 = (System.nanoTime - t2) / 1e9d
duration2

23/08/20 14:17:43 INFO CodeGenerator: Code generated in 19.777958 ms
23/08/20 14:17:43 INFO DAGScheduler: Registering RDD 39 (show at cell16.sc:6) as input to shuffle 2
23/08/20 14:17:43 INFO DAGScheduler: Got map stage job 7 (show at cell16.sc:6) with 44 output partitions
23/08/20 14:17:43 INFO DAGScheduler: Final stage: ShuffleMapStage 9 (show at cell16.sc:6)
23/08/20 14:17:43 INFO DAGScheduler: Parents of final stage: List()
23/08/20 14:17:43 INFO DAGScheduler: Missing parents: List()
23/08/20 14:17:43 INFO DAGScheduler: Submitting ShuffleMapStage 9 (MapPartitionsRDD[39] at show at cell16.sc:6), which has no missing parents
23/08/20 14:17:43 INFO MemoryStore: Block broadcast_10 stored as values in memory (estimated size 47.4 KiB, free 136.0 MiB)
23/08/20 14:17:43 INFO MemoryStore: Block broadcast_10_piece0 stored as bytes in memory (estimated size 21.6 KiB, free 136.0 MiB)
23/08/20 14:17:43 INFO BlockManagerInfo: Added broadcast_10_piece0 in memory on 192.168.88.21:62937 (size: 21.6

+----------+--------------------+
|content_id|          avg(month)|
+----------+--------------------+
|      4519|0.015429677931821352|
|      4818|0.016230649272587192|
|      5518| 0.02103952664360606|
|     13285| 0.04141825984987046|
|     12027| 0.04150963056316861|
|       833|0.022584471034752644|
|      9427| 0.02158891106034129|
|       496|0.020155351740181845|
|      5156|0.018064186032779997|
|      6336|0.019195051162991618|
|      2866|0.026921431007821156|
|      1959| 0.02670915106444127|
|      7982| 0.02122819619071833|
|     23336|0.019260306662998706|
|       148|0.017910791660268703|
|      1342| 0.02195320830526077|
|      1088| 0.02461949633696784|
|      1580|0.022461073541947943|
|       471| 0.02611517946736722|
|      2122|0.026057020501248997|
+----------+--------------------+
only showing top 20 rows



t2: Long = 1652196022026125L
duration2: Double = 2.947987458
res16_3: Double = 2.947987458

In [17]:
println(duration1, duration2)

(5.363646375,2.947987458)
